In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

# Import functions
from cmdstanpy import CmdStanModel
from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions

# Create ./stan folder if does not exists
if not os.path.exists("./stan"):
    os.mkdir("./stan")
!pip install --upgrade cmdstanpy
import cmdstanpy
cmdstanpy.install_cmdstan()

In [ ]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
import numpy as np
import jax
!pip install mycolorpy

# **Data** **Generation**

In [ ]:
from IPython.utils.sysinfo import num_cpus

# numero di cluster
K=5

# dimensione del campione
d=2

# numero di sample
N=1000

seed=2022
key = random.PRNGKey(seed)
# vectors of mean of clusters
sigma=5
mu=random.normal(key,(K,d))*sigma

# cluster assignment
key = random.PRNGKey(2)
c=random.categorical(key,(1/K)*jnp.ones(shape=(K,)),axis=0,shape=(N,))
C=np.zeros(shape=(N,K))
for i in range(N):
  C[i,c[i]]=1
C=jnp.array(C)

# Data
X=jnp.matmul(C,mu)+random.normal(key,(N,d))
import matplotlib.pyplot as plt

for i in range(K):
  plt.scatter(X[c==i,0],X[c==i,1])
plt.show()

Implementation of MCMC in STAN(based on hamiltonian MC)

In [ ]:
gaussian_mix_uni = """
data {
  int N;
  int H;
  array[N,2] real y;
  array[2] real p0_params;
}

transformed data {
  real mu0 = p0_params[1];
  real sigma0 = p0_params[2];
 }

parameters {
  array[H] real means;
  array[H] real <lower=0> std_devs;
  simplex[H] weights;
 }


model {

   //Likelihood
   vector[H] contributions;
   for (i in 1:N){
     for (h in 1:H){
      contributions[h] = log(weights[h]) + normal_lpdf(y[i] | means[h], std_devs[h]);
      }
      target += log_sum_exp(contributions);
    }


   // Prior
   means ~ normal(mu0, sigma0);
   std_devs ~ uniform(0,10);
   //weights ~ categorical(rep_array(1,H));

}

generated quantities{
  }
"""
# Write stan model to file
stan_file = "./stan/gaussian_mix_uni.stan"
with open(stan_file, "w") as f:
    print(gaussian_mix_uni, file=f)

# Compile stan model
gaussian_mix_uni = CmdStanModel(stan_file=stan_file)

In [ ]:
# Input data
gmm_data = {
    "N": N,
    "H": 4,
    "y": X.tolist(),
    "p0_params": np.array([0, 4]).tolist()
}

# Algorithm parameters
algo_params = {
    "n_chains": 4,
    "n_burnin": 1000,
    "n_iter": 1000
}

# Sample
gmm_fit = gaussian_mix_uni.sample(data=gmm_data, chains=algo_params["n_chains"], parallel_chains=algo_params["n_chains"],
                                  iter_warmup=algo_params["n_burnin"], iter_sampling=algo_params["n_iter"])
gmm_fit
# Convert to arviz data type
#chains = az.from_cmdstanpy(gmm_fit)

In [52]:
from google.colab import files

# Access the output files
csv_files = gmm_fit.runset.csv_files
output_files = gmm_fit.runset.stdout_files

# Download CSV files
for csv_file in csv_files:
    print(f"Downloading CSV file: {csv_file}")
    files.download(csv_file)

# Download stdout files
for output_file in output_files:
    print(f"Downloading output file: {output_file}")
    files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>